

>baseline





將關鍵字比對換成向量相似度比對。

請將目前使用關鍵字比對的 route_by_query，改為使用向量相似度進行分類，並設一個合理的相似度門檻，根據檢索結果的分數判斷是否走 RAG 流程。 例如用向量相似度及自訂 threshold 決定要不要分到 retriever。

Hint：similarity_search_with_score(...) 可參考去年的讀書會 R4：向量資料庫的基本操作


In [1]:
!pip install -U langchain-community
!pip install -U langchain langchain-community langchain-huggingface chromadb

In [2]:
# 1. 匯入套件
from langchain_core.documents import Document
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

# 2. 定義文本資料（火影人物表）
docs_text = """
火影代數\t姓名\t師傅\t徒弟
初代\t千手柱間\t無明確記載\t猿飛日斬、水戶門炎、轉寢小春
二代\t千手扉間\t千手柱間（兄長）\t猿飛日斬、志村團藏、宇智波鏡等
三代\t猿飛日斬\t千手柱間、千手扉間\t自來也、大蛇丸、千手綱手（傳說三忍）
四代\t波風湊\t自來也\t旗木卡卡西、宇智波帶土、野原琳
五代\t千手綱手\t猿飛日斬\t春野櫻、志乃等（主要為春野櫻）
六代\t旗木卡卡西\t波風湊\t漩渦鳴人、宇智波佐助、春野櫻（第七班）
七代\t漩渦鳴人\t自來也、旗木卡卡西\t木葉丸等（主要為木葉丸）
"""

# 3. 拆成多筆 Document（每一段為一筆）
docs = [Document(page_content=txt.strip()) for txt in docs_text.strip().split("\n")]

# 4. 建立 embedding 模型（支援中文）
embedding_model = HuggingFaceEmbeddings(
    model_name="infgrad/stella-base-zh-v3-1792d",
    encode_kwargs={"normalize_embeddings": True}
)

# 5. 建立向量資料庫（使用 Chroma）
vectorstore = Chroma.from_documents(
    documents=docs,
    embedding=embedding_model,
    collection_name="naruto_collection",
    persist_directory="./naruto_store"
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertModel were not initialized from the model checkpoint at infgrad/stella-base-zh-v3-1792d and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/5.51M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

In [3]:
##### 查詢 + 判斷是否進入 RAG #####

def route_by_similarity(query: str, threshold: float = 0.65):
    # 查詢最相似的資料
    results = vectorstore.similarity_search_with_score(query, k=1)

    if not results:
        print("查無結果，直接交給模型回答")
        return "direct"

    top_doc, score = results[0]
    print(f"相似度分數（越小越像）：{score:.3f}")
    print(f"相關內容：{top_doc.page_content}\n")

    if score < threshold:
        return "rag"
    else:
        return "direct"

# ✅ 示範使用者互動
while True:
    user_input = input("使用者：")
    if user_input.lower() in ["q", "quit", "exit"]:
        break

    path = route_by_similarity(user_input)

    if path == "rag":
        print("→ 進入 RAG 檢索流程（用資料輔助模型回答）")
    else:
        print("→ 不進入 RAG，交給模型自己回答")

    print("===" * 20)


使用者：誰是第四代火影?
相似度分數（越小越像）：0.604
相關內容：四代	波風湊	自來也	旗木卡卡西、宇智波帶土、野原琳

→ 進入 RAG 檢索流程（用資料輔助模型回答）
使用者：第四代火影的師傅是誰?
相似度分數（越小越像）：0.472
相關內容：火影代數	姓名	師傅	徒弟

→ 進入 RAG 檢索流程（用資料輔助模型回答）
使用者： 第四代火影的徒弟有哪些人?
相似度分數（越小越像）：0.483
相關內容：火影代數	姓名	師傅	徒弟

→ 進入 RAG 檢索流程（用資料輔助模型回答）
使用者： 相對論是誰發明的?
相似度分數（越小越像）：1.295
相關內容：七代	漩渦鳴人	自來也、旗木卡卡西	木葉丸等（主要為木葉丸）

→ 不進入 RAG，交給模型自己回答
使用者：q




> advance


改成能支援多輪問答（Multi-turn RAG），並能根據前面的query判斷問題。

請將 RAGState 加入 history 欄位，並在生成回答時，將歷史對話與當前問題一起組成 prompt。

Hint：

class MultiTurnRAGState(TypedDict):  
    history: List[str]  
    query: str  
    docs: List[Document]  
    answer: str

In [2]:
# ✅ 套件安裝指令（Colab 使用）
!pip install -U langchain langchain-community langchain-huggingface chromadb langgraph transformers

In [4]:
# ✅ 向量知識庫建立
from langchain_core.documents import Document
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

# 火影資料表
docs_text = """
火影代數	姓名	師傅	徒弟
初代	千手柱間	無明確記載	猿飛日斬、水戶門炎、轉寢小春
二代	千手扉間	千手柱間（兄長）	猿飛日斬、志村團藏、宇智波鏡等
三代	猿飛日斬	千手柱間、千手扉間	自來也、大蛇丸、千手綱手（傳說三忍）
四代	波風湊	自來也	旗木卡卡西、宇智波帶土、野原琳
五代	千手綱手	猿飛日斬	春野櫻、志乃等（主要為春野櫻）
六代	旗木卡卡西	波風湊	漩渦鳴人、宇智波佐助、春野櫻（第七班）
七代	漩渦鳴人	自來也、旗木卡卡西	木葉丸等（主要為木葉丸）
"""

lines = docs_text.strip().split("\n")[1:]  # 去掉標題
docs = []
for line in lines:
    generation, name, master, students = line.split("\t")
    content = f"{generation}火影是{name}，師父是{master}，徒弟是{students}。"
    docs.append(Document(page_content=content))

embedding_model = HuggingFaceEmbeddings(
    model_name="infgrad/stella-base-zh-v3-1792d",
    encode_kwargs={"normalize_embeddings": True}
)

vectorstore = Chroma.from_documents(
    documents=docs,
    embedding=embedding_model,
    collection_name="naruto_collection",
    persist_directory="./naruto_store"
)

# ✅ 載入 Qwen Chat 模型
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_id = "Qwen/Qwen1.5-0.5B-Chat"
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True, device_map="auto")

generator = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.4,
    return_full_text=False
)

# ✅ 將多輪對話包成 Qwen 格式的 chat prompt
def build_chat_prompt(history: list, query: str):
    messages = [{"role": "user", "content": h} for h in history]
    messages.append({"role": "user", "content": query})
    messages.append({"role": "assistant", "content": ""})
    return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

# ✅ 定義 State 結構
from typing_extensions import TypedDict, List
from langchain_core.documents import Document

class MultiTurnRAGState(TypedDict):
    history: List[str]
    query: str
    docs: List[Document]
    answer: str

# ✅ 定義節點

def retrieve_node(state: MultiTurnRAGState) -> MultiTurnRAGState:
    docs = vectorstore.similarity_search(state["query"], k=3)
    return {**state, "docs": docs}

def generate_node(state: MultiTurnRAGState) -> MultiTurnRAGState:
    context = "\n".join(doc.page_content for doc in state["docs"])
    history_prompt = f"根據以下知識回答問題：\n{context}"
    prompt = build_chat_prompt(state["history"] + [history_prompt], state["query"])
    output = generator(prompt, max_new_tokens=200)[0]["generated_text"]
    return {**state, "answer": output, "history": state["history"] + [state["query"], output]}

def direct_generate_node(state: MultiTurnRAGState) -> MultiTurnRAGState:
    prompt = build_chat_prompt(state["history"], state["query"])
    output = generator(prompt, max_new_tokens=200)[0]["generated_text"]
    return {**state, "answer": output, "history": state["history"] + [state["query"], output]}

def reject_node(state: MultiTurnRAGState) -> MultiTurnRAGState:
    prompt = build_chat_prompt(state["history"], state["query"])
    output = generator(prompt, max_new_tokens=200)[0]["generated_text"]
    return {**state, "answer": output, "history": state["history"] + [state["query"], output]}

# ✅ 條件判斷節點

def is_related_to_naruto(query: str) -> bool:
    keywords = ["火影", "忍者", "卡卡西", "木葉", "鳴人", "自來也", "綱手", "帶土", "琳"]
    return any(k in query for k in keywords)

from langchain_core.runnables import RunnableLambda

def route_by_query(sqtate: MultiTurnRAGState) -> str:
    results = vectorstore.similarity_search_with_score(state["query"], k=3)
    avg_score = sum(score for _, score in results) / len(results)
    print(f"→ 相似度 {avg_score:.3f}")
    if avg_score > 1.0:
        return "reject"
    elif avg_score < 0.8 and is_related_to_naruto(state["query"]):
        return "rag"
    else:
        return "direct"

# ✅ LangGraph 組裝
from langgraph.graph import StateGraph, END

graph_builder = StateGraph(MultiTurnRAGState)
graph_builder.set_entry_point("condition")
graph_builder.add_node("condition", RunnableLambda(lambda x: x))
graph_builder.add_node("retriever", RunnableLambda(retrieve_node))
graph_builder.add_node("generator", RunnableLambda(generate_node))
graph_builder.add_node("direct_generator", RunnableLambda(direct_generate_node))
graph_builder.add_node("reject", RunnableLambda(reject_node))

graph_builder.add_conditional_edges(
    "condition", RunnableLambda(route_by_query),
    {"rag": "retriever", "direct": "direct_generator", "reject": "reject"}
)
graph_builder.add_edge("retriever", "generator")
graph_builder.add_edge("generator", END)
graph_builder.add_edge("direct_generator", END)
graph_builder.add_edge("reject", END)

graph = graph_builder.compile()

# ✅ 開始對話
history = []
print("開始對話吧（輸入 q 結束）")
while True:
    user_input = input("使用者: ")
    if user_input.lower() in ["q", "quit", "exit"]:
        print("掰啦！")
        break
    state = {"history": history, "query": user_input, "docs": [], "answer": ""}
    result = graph.invoke(state)
    print("AI 助理：", result["answer"].strip())
    print("=" * 60)
    history = result["history"]


Some weights of BertModel were not initialized from the model checkpoint at infgrad/stella-base-zh-v3-1792d and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0


開始對話吧（輸入 q 結束）
使用者: 第四代火影是誰?
→ 相似度 0.340
AI 助理： 第四代火影是波風湊，師父是自來也，徒弟是旗木卡卡西、宇智波帶土、野原琳。
使用者: 他的師父是誰?
→ 相似度 0.772
AI 助理： 第四代火影的師父是自來也。
使用者: 他的徒弟有哪些人?
→ 相似度 0.724
AI 助理： 第四代火影的徒弟有：

1. 旗木卡卡西：他是第四代火影的主角，也是团队的主要成员之一。
2. 宇智波带土：他是第四代火影的辅助角色，同时也是团队的重要支持者。
3. 尼古拉·风影：她是第四代火影的伙伴，也是团队中的重要成员之一。

第四代火影的其他主要人物包括：

1. 鲁智深：他是第四代火影的队友，也是团队中的重要成员之一。
2. 松下忍者：他是第四代火影的导师，也是团队中的重要成员之一。
3. 神崎勇者：他是第四代火影的同伴，也是团队中的重要成员之一。
使用者: 相對論是他發明的嗎?
→ 相似度 1.277
AI 助理： 是的，相對論是由日本作家村上春樹在《火影忍者》中提出的，他是在第一卷「火影忍者」中提出的概念，后来被广泛接受并融入到其他作品中。相對論的概念强调了火影忍者的身份和背景，以及他们与其他角色之间的关系，从而构建了一个更加复杂而立体的角色世界。
使用者: 相對論不是愛因斯坦嗎?
→ 相似度 1.372
AI 助理： 不，相對論并不是愛因斯坦提出的。相對論是由日本作家村上春樹在《火影忍者》中提出的概念，他在第一卷「火影忍者」中提出了这个概念，并且后来被广泛接受并融入到其他作品中。相對論的概念强调了火影忍者的身份和背景，以及他们与其他角色之间的关系，从而构建了一个更加复杂而立体的角色世界。
使用者: q
掰啦！
